In [1]:
import json
import pandas as pd
import requests
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py

### Functions for working with SQLite

In [2]:
def create_database(db_name):
    """ 
    create an SQLite database
    """
    conn = None
    try:
        conn = db.connect(db_name)
        #print(db.version)
    except Error as e:
        print(e)
    finally:
        conn.close()
            
def create_connection(db_name):
    """ 
    create a database connection to the SQLite database
        specified by db_name
    return: Connection object or None
    """
    conn = None
    try:
        conn = db.connect(db_name)
        return conn
    except Error as e:
        print(e)

    return conn


def check_table_exists(tbl_name, db_name): 
    """
    Check if table exists
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    exists = 0
    c.execute("SELECT count(name) FROM sqlite_master WHERE type='table'")
    if c.fetchone()[0] == 0:
        print("No tables in DB.")
    else:
        try:
            c.execute("SELECT COUNT(name) FROM sqlite_master WHERE type = 'table' AND name='" + seriesID+"';")
        except Error as e:
            print(e)
        if c.fetchone()[0] == 1:
            exists = 1
        
    conn.close()
    
    return exists 
    
    
def create_table(input_df, tbl_name, db_name):
    """
    create table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    if not check_table_exists(tbl_name ,db_name):
        try: 
            input_df.to_sql(tbl_name, conn)
            print("Created ", tbl_name, " table.")
        except ValueError as e:
            print(e)
            print("Table was not added to the DB")       
    conn.close()
    
    
def validate_df(df, tbl_name, db_name):
    """
    check if the data from the api df already exists in the table and 
    if the data in the table is duplicate or inconsistent
    return dataframe df without duplicate data or inconsistent data
    return dataframe df_dup_data with duplicate date
    return dataframe df_inconsistent_data with inconsistent data
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    df_not_dup_data = pd.DataFrame(columns = list(df.columns))
    df_dup_data = pd.DataFrame(columns = list(df.columns))
    df_inconsistent_data = pd.DataFrame(columns = list(df.columns))
    
    if check_table_exists(seriesID, db_name) != 0:
        # Check if data already exists in DB
        for i, row in df.iterrows():
            c.execute("SELECT date, value FROM " + tbl_name + " WHERE date = ?;", (df.loc[i]["date"],))
            queryOut = c.fetchall()
            
            # Data does not already exist in the database 
            if len(queryOut) == 0: 
                df_not_dup_data = pd.concat([df_not_dup_data, df.loc[[i]]])
            # Data already exists in the database with duplicate values
            elif queryOut[0][1] == df.loc[i]["value"]: 
                df_dup_data = pd.concat([df_dup_data, df.loc[[i]]])
            # Data already exists in the database but with different values
            elif queryOut[0][1] != df.loc[i]["value"]:
                df_inconsistent_data = pd.concat([df_inconsistent_data, df.loc[[i]]])            
    else:
        df_not_dup_data = df.copy()
    
    conn.close()
    
    df_not_dup_data = df_not_dup_data.sort_values(by=['date'])
    
    return df_not_dup_data, df_dup_data, df_inconsistent_data


def add_to_database(df_EntryData, tbl_name, db_name, append_replace):  
    """
    Add dataframe to 
    database: db_name 
    table: tbl_name
    """
    conn = create_connection(db_name)    
    try:
        df_EntryData.to_sql(seriesID, con=conn, if_exists = append_replace)
        print("Data added to DB: \n", df_EntryData)
    except ValueError as e:
        print(e)
        print("Data was not added to the DB")
    conn.close()
    
    
def get_max_index(tbl_name, db_name):
    '''
    Get the maximum index value of the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT MAX([index]) FROM " + seriesID)
        max_index = c.fetchone()[0]
        return max_index
    except Error as e:
        print(e)
    finally:
        conn.close()
        
def get_max_date(tbl_name, db_name):
    '''
    Get the maximum date from the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT date FROM " + seriesID + " WHERE [index] = (SELECT MAX([index]) FROM " + seriesID +")")
        max_date = c.fetchone()[0]
        return max_date
    except Error as e:
        print(e)
    finally:
        conn.close()

def table_to_df(tbl_name, db_name):
    conn = db.connect(db_name)
    try:
        df_fromDB = pd.read_sql_query("SELECT * FROM " + tbl_name + ";", conn, index_col="index")
    except Error as e:
        print(e)
    conn.close()
    
    return df_fromDB

### Get data from BLS

In [3]:
def get_BLS_data(seriesID, startyear, endyear):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = {
        "seriesid":seriesIDs, 
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        message = "For series " + seriesID + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [4]:
# show all the rows in the dataframe only use this for testing
# pd.set_option('display.max_rows', None)

### Create and Format Dataframe

In [5]:
def format_df_from_json(data, description):
    # Create dataframe from JSON
    #df = pd.DataFrame(json_data['Results']['series'][0]['data'])
    df = pd.DataFrame(data)

    # Format date column and sort by date
    df['date'] = df['year'] + df['period']
    df['date'] = df['date'].str.replace('M', '-')
    df['date'] = df['date'].astype('string')
    df = df.sort_values(by=['date'], ignore_index=True)

    # Set the index using the value from the max index in the db table
    df["index"] = range(1, len(df.index) +1 )
    df = df.set_index(df["index"])

    df = df.drop(columns=['footnotes', 'index'])
    df['value'] = df['value'].astype(float)
    df['description'] = description
    df['date_updated'] = datetime.now()
    df['date_updated'] = df['date_updated'].astype('string')
    
    return df

In [6]:
def align_df_db_indexes(df, seriesID, db_name):
    # Concat the data from the database table and the dataframe, then sort by date
    df_db = table_to_df(seriesID, db_name)
    df = pd.concat([df, df_db])
    df = df.sort_values(by=['date'], ignore_index=True)

    return df

In [7]:
# CUUR0000SA0: 
# CUSR0000SA0
seriesIDs = ["CUUR0000SA0", "CUSR0000SA0"] 
db_name = "MacroData.sqlite3"
message, json_data = get_BLS_data(seriesIDs, 2003, 2022)
if message:
    print(message)

In [8]:
for i in range(len(seriesIDs)):
    seriesID = (json_data['Results']['series'][i]['seriesID'])
    data = (json_data['Results']['series'][i]['data'])
    description = json_data['Results']['series'][i]['catalog']['survey_name'] + ', ' + json_data['Results']['series'][i]['catalog']['series_title']
    print(seriesID)
    print(description)

    df = format_df_from_json(data, description)

    df, df_dup_data, df_inconsistent_data = validate_df(df, seriesID, db_name)

    # Deal with duplicate data
    if not df_dup_data.empty:
        show_dups = input("Do you want to see the duplicate data? Y/N ").upper()
        if show_dups == "Y":
            print(df_dup_data)
    else:
        del df_dup_data

    # Deal with inconsistent data
    if not df_inconsistent_data.empty:
        show_incons = input("Do you want to see the inconsistent data? Y/N ").upper()
        if show_incons == "Y":
            print(df_inconsistent_data)
    else:
        del df_inconsistent_data

    if check_table_exists(seriesID, db_name) != 0:
        df = align_df_db_indexes(df, seriesID, db_name)
        add_to_database(df, seriesID, db_name, 'replace')
    else:
        add_to_database(df, seriesID, db_name, 'append')

CUUR0000SA0
CPI for All Urban Consumers (CPI-U), All items in U.S. city average, all urban consumers, not seasonally adjusted
No tables in DB.
No tables in DB.
Data added to DB: 
        year period periodName latest    value     date  \
index                                                    
1      2003    M01    January    NaN  181.700  2003-01   
2      2003    M02   February    NaN  183.100  2003-02   
3      2003    M03      March    NaN  184.200  2003-03   
4      2003    M04      April    NaN  183.800  2003-04   
5      2003    M05        May    NaN  183.500  2003-05   
...     ...    ...        ...    ...      ...      ...   
232    2022    M04      April    NaN  289.109  2022-04   
233    2022    M05        May    NaN  292.296  2022-05   
234    2022    M06       June    NaN  296.311  2022-06   
235    2022    M07       July    NaN  296.276  2022-07   
236    2022    M08     August   true  296.171  2022-08   

                                             description  \
index

In [11]:
conn = create_connection(db_name)
c = conn.cursor()
try:
    c.execute("SELECT name FROM sqlite_schema WHERE type ='table' AND name NOT LIKE 'sqlite_%'")
    output = c.fetchall()
    print(output)
except Error as e:
    print(e)
finally:
    conn.close()

[('CUUR0000SA0',), ('CUSR0000SA0',)]


In [11]:
df = format_df_from_json(json_data)
df

,year,period,periodName,latest,value,date,description,date_updated
index,,,,,,,,
1,2003,M01,January,NaN,181.700,2003-01,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
2,2003,M02,February,NaN,183.100,2003-02,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
3,2003,M03,March,NaN,184.200,2003-03,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
4,2003,M04,April,NaN,183.800,2003-04,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
5,2003,M05,May,NaN,183.500,2003-05,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
...,...,...,...,...,...,...,...,...
232,2022,M04,April,NaN,289.109,2022-04,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
233,2022,M05,May,NaN,292.296,2022-05,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083
234,2022,M06,June,NaN,296.311,2022-06,"CPI for All Urban Consumers (CPI-U), All items...",2022-09-26 18:49:53.285083


In [10]:
df = format_df_from_json(json_data)
df, df_dup_data, df_inconsistent_data = validate_df(df, seriesID, db_name)
df

NameError: name 'seriesID' is not defined

In [27]:
# Deal with duplicate data
if not df_dup_data.empty:
    show_dups = input("Do you want to see the duplicate data? Y/N").upper()
    if show_dups == "Y":
        print(df_dup_data)
else:
    del df_dup_data
    
# Deal with inconsistent data
if not df_inconsistent_data.empty:
    show_incons = input("Do you want to see the inconsistent data? Y/N ").upper()
    if show_incons == "Y":
        print(df_inconsistent_data)
else:
    del df_inconsistent_data
    
if check_table_exists(seriesID, db_name) != 0:
    df = align_df_db_indexes(df, seriesID, db_name)
    add_to_database(df, seriesID, db_name, 'replace')
else:
    add_to_database(df, seriesID, db_name, 'append')
#df

Do you want to see the duplicate data? Y/Ny
     year period periodName value     date  \
121  1923    M01    January  16.8  1923-01   
122  1923    M02   February  16.8  1923-02   
123  1923    M03      March  16.8  1923-03   
124  1923    M04      April  16.9  1923-04   
125  1923    M05        May  16.9  1923-05   
126  1923    M06       June  17.0  1923-06   
127  1923    M07       July  17.2  1923-07   
128  1923    M08     August  17.1  1923-08   
129  1923    M09  September  17.2  1923-09   
130  1923    M10    October  17.3  1923-10   
131  1923    M11   November  17.3  1923-11   
132  1923    M12   December  17.3  1923-12   
133  1924    M01    January  17.3  1924-01   
134  1924    M02   February  17.2  1924-02   
135  1924    M03      March  17.1  1924-03   
136  1924    M04      April  17.0  1924-04   
137  1924    M05        May  17.0  1924-05   
138  1924    M06       June  17.0  1924-06   
139  1924    M07       July  17.1  1924-07   
140  1924    M08     August  17.0  1

In [17]:
df.dtypes

year            object
period          object
periodName      object
value           object
date            object
description     object
date_updated    object
dtype: object

In [28]:
df_cpi_not_seasonal = table_to_df(seriesID, db_name)
df_cpi_not_seasonal

,year,period,periodName,value,date,description,date_updated
index,,,,,,,
0,1913,M01,January,9.8,1913-01,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
1,1913,M02,February,9.8,1913-02,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
2,1913,M03,March,9.8,1913-03,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
3,1913,M04,April,9.8,1913-04,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
4,1913,M05,May,9.7,1913-05,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
5,1913,M06,June,9.8,1913-06,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
6,1913,M07,July,9.9,1913-07,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
7,1913,M08,August,9.9,1913-08,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268
8,1913,M09,September,10.0,1913-09,"CPI for All Urban Consumers (CPI-U), All items...",2022-08-30 16:09:31.147268


In [15]:
#con = db.connect('MacroData.sqlite3')
#c = con.cursor()
#c.execute("DROP TABLE " + 'CUUR0000SA0')
#con.commit()
#con.close()

OperationalError: no such table: CUUR0000SA0